 # Comprehensive Exam

 ## [Coding Artifact](./00-kg-main-artifact.ipynb)

 Kalin Gibbons

 Nov 20, 2020
 

 # Model Selection

 Base selection of regressors is performed by fitting multiple regressors
 without any prior hyperparameter tuning, then comparing the resulting errors
 across functional groups. Models with lower errors will be marked for
 parameter tuning investigations.

In [1]:
import os
import sys
import math
import logging
from pathlib import Path
from IPython.display import display

import numpy as np

import sklearn
from sklearn.ensemble import (
    AdaBoostRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor
)
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from tqdm.auto import tqdm


%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# import seaborn as sns
import pandas as pd

import artifact
from artifact.datasets import load_tkr, tkr_group_lut
from artifact.helpers import RegressionProfile, REGRESSION_PROFILE_PATH



In [2]:
plt.rcParams['figure.figsize'] = (9, 5.5)
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.size'] = 14
mpl.rcParams['font.family'] = 'Times New Roman'

# sns.set_context("poster")
# sns.set(rc={'figure.figsize': (16, 9.)})
# sns.set_style("whitegrid")

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)



 ## Profiling the regressors

 First, we'll choose potential regressors to investigate. Early choices are
 linear, decision trees, as well as boosting and forest ensemble methods.
 ### Learner Selection

In [3]:
learners = (
    GradientBoostingRegressor(n_estimators=100),
    RandomForestRegressor(n_estimators=100),
    AdaBoostRegressor(DecisionTreeRegressor(), n_estimators=100),
    AdaBoostRegressor(LinearRegression(), n_estimators=100),
    DecisionTreeRegressor(),
    LinearRegression()
)


 Next, we'll select a functional group to examine, and only load the necessary
 data.
 ### Functional group selection

In [4]:
func_groups = list(tkr_group_lut.keys())
func_groups


['contact_mechanics', 'joint_loads', 'kinematics', 'ligaments', 'patella']

In [5]:
group = 'patella'


 ### Loading the data

 We'll load a subset of the data containing the responses making up the chosen
 functional group. We'll also use a `RegressionProfile` object to allow
 persistent results.

In [6]:
shared_kwargs = dict(load_fcn=load_tkr, functional_group=group)
tkr_train = artifact.Results(**shared_kwargs, subset='train')
tkr_test = artifact.Results(**shared_kwargs, subset='test')
display(tkr_train.response_names[1:])

reg_prof = RegressionProfile(load_path=REGRESSION_PROFILE_PATH)


['patella_area',
 'pat_cop_1',
 'pat_cop_2',
 'pat_cop_3',
 'pat_press',
 'pat_force_1',
 'pat_force_2',
 'pat_force_3',
 'pl_disp',
 'pl_force',
 'pat_fem_flexion',
 'pat_fem_valgus',
 'pat_fem_external',
 'pat_fem_lat',
 'pat_fem_ant',
 'pat_fem_inf']

 ### Fitting and profiling
 If the profiling results from the selected functional group have been loaded,
 then the `force_search` flag will need to be set to `True` to overwrite the
 previous profiling session.

In [7]:
force_search = False


In [8]:
learner_names = [x.__str__().replace('()', '') for x in learners]
scaler = StandardScaler()
regr = artifact.Regressor(tkr_train, tkr_test, learners[0], scaler=scaler)
err_df = pd.DataFrame(index=learner_names)

saved_keys = reg_prof.error_dataframes.keys()
if (force_search) or (group not in saved_keys):
    resp_pbar = tqdm(regr.train_results.response_names, desc='Processing...')
    for resp in resp_pbar:
        if resp == 'time':
            continue
        resp_pbar.set_description(f'Processing {resp}')
        errs = np.zeros_like(learner_names, dtype=np.float)
        lrn_pbar = tqdm(learners, desc='Fitting...', leave=False)
        for idx, lrn in enumerate(lrn_pbar):
            desc = f'{learner_names[idx].replace("base_estimator=", "")}'
            lrn_pbar.set_description(desc)
            regr.learner = MultiOutputRegressor(lrn)
            y_pred = regr.fit(resp).predict()
            errs[idx] = regr.prediction_error
        err_df[resp] = errs
        lrn_pbar.close()
    resp_pbar.close()

    reg_prof.add_results(group, err_df)
    reg_prof.save(REGRESSION_PROFILE_PATH)


 ## Results

In [9]:
# reg_prof.summarize(group)
for key in reg_prof.error_dataframes.keys():
    reg_prof.summarize(key)




joint_loads
-----------

Best learners total by response:


LinearRegression                                                             4
AdaBoostRegressor(base_estimator=DecisionTreeRegressor, n_estimators=100)    2
GradientBoostingRegressor                                                    2
dtype: int64

med_torque_1    AdaBoostRegressor(base_estimator=DecisionTreeR...
lat_torque_1    AdaBoostRegressor(base_estimator=DecisionTreeR...
med_force_2                             GradientBoostingRegressor
lat_force_2                             GradientBoostingRegressor
med_force_1                                      LinearRegression
med_torque_2                                     LinearRegression
lat_force_1                                      LinearRegression
lat_torque_2                                     LinearRegression
dtype: object



Sorted by median RMS error (smallest to largest):


,count,mean,std,min,25%,50%,75%,max
"AdaBoostRegressor(base_estimator=DecisionTreeRegressor, n_estimators=100)",8.0,457.436336,615.357405,33.018228,62.568494,175.478336,557.412198,1486.255491
RandomForestRegressor,8.0,465.865525,624.637692,32.185607,65.538200,180.323926,567.866601,1514.090976
GradientBoostingRegressor,8.0,453.882096,605.045488,28.624252,59.692740,181.805110,561.551478,1471.967865
LinearRegression,8.0,447.428664,586.178902,33.169663,55.411237,185.267365,565.693107,1430.352219
"AdaBoostRegressor(base_estimator=LinearRegression, n_estimators=100)",8.0,537.220447,681.397757,36.384291,69.945561,234.843365,709.332590,1691.947807
DecisionTreeRegressor,8.0,669.671069,891.333514,47.775067,104.640293,254.695111,823.570689,2173.725325


...

ap_disp      LinearRegression
pmc_disp     LinearRegression
pom_disp     LinearRegression
pol_disp     LinearRegression
pcl_disp     LinearRegression
pcm_disp     LinearRegression
alc_disp     LinearRegression
vi1_disp     LinearRegression
pfl_force    LinearRegression
lclp_disp    LinearRegression
lcla_disp    LinearRegression
lcl_disp     LinearRegression
mclp_disp    LinearRegression
mcla_disp    LinearRegression
mcl_disp     LinearRegression
pfl_disp     LinearRegression
vi1_force    LinearRegression
dtype: object



Sorted by median RMS error (smallest to largest):


,count,mean,std,min,25%,50%,75%,max
LinearRegression,17.0,2.644678,6.578398,0.496801,0.690125,0.857190,0.982690,27.809719
"AdaBoostRegressor(base_estimator=LinearRegression, n_estimators=100)",17.0,3.071228,7.433703,0.533512,0.762196,1.032230,1.193374,31.357367
GradientBoostingRegressor,17.0,3.523874,8.862122,0.595402,0.922500,1.142407,1.385996,37.667473
"AdaBoostRegressor(base_estimator=DecisionTreeRegressor, n_estimators=100)",17.0,3.898217,9.532387,0.639053,1.085326,1.327939,1.643020,40.645751
RandomForestRegressor,17.0,4.051685,10.089778,0.655742,1.105437,1.344097,1.649689,42.980346
DecisionTreeRegressor,17.0,5.993554,14.507311,1.057411,1.711025,2.147110,2.661122,61.997250




RMS Errors:


,ap_disp,mcl_disp,mcla_disp,mclp_disp,lcl_disp,lcla_disp,lclp_disp,pfl_disp,pfl_force,alc_disp,pcm_disp,pcl_disp,pol_disp,pom_disp,pmc_disp,vi1_disp,vi1_force
GradientBoostingRegressor,1.320910,0.866712,0.833466,0.895061,1.302893,1.360081,1.385996,1.834739,37.667473,1.142407,1.026687,0.924339,1.078023,0.922500,1.415725,0.595402,5.333449
RandomForestRegressor,1.649689,1.079333,1.045374,1.105437,1.545818,1.617806,1.639246,2.155293,42.980346,1.326170,1.178683,1.035248,1.344097,1.218054,1.717247,0.655742,5.585059
"AdaBoostRegressor(base_estimator=DecisionTreeRegressor, n_estimators=100)",1.598345,1.054265,1.020414,1.085326,1.581645,1.643020,1.682200,2.164975,40.645751,1.327939,1.140784,1.041014,1.309432,1.127109,1.605072,0.639053,5.603342
"AdaBoostRegressor(base_estimator=LinearRegression, n_estimators=100)",1.234940,0.659524,0.643355,0.670545,1.050308,1.068814,1.032230,1.193374,31.357367,1.286204,0.890204,0.998838,1.101196,0.762196,0.840768,0.533512,6.887504
DecisionTreeRegressor,2.596926,1.671426,1.597184,1.711025,2.589422,2.643307,2.671249,3.263703,61.997250,2.147110,1.848993,1.668983,2.145812,1.806551,2.661122,1.057411,7.812937
LinearRegression,1.095271,0.527545,0.515668,0.534548,0.837581,0.857190,0.867430,1.092973,27.809719,0.901664,0.838817,0.876701,0.982690,0.690125,0.744542,0.496801,5.290260


Return to the root [Coding Artifact](./00-kg-main-artifact.ipynb) document.